# Functions for running training loops

## K-Fold Training and Cross-Validation

In [ ]:
#default_exp trainers
#export 
import os

import torch
import pandas as pd
import pytorch_lightning as lit
import wandb
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

from reappraisalmodel.lightningreapp import LightningReapp

# Default configuration for hyperparameters
default_config = {
    'lr': 1e-3,
    'hidden_layer_size': 50
    }

def kfold_train(k: int, ldhdata, strat='far', **trainer_kwargs) -> None:
    """Fits a LightningReapp instance with k-fold cross-validation.
    Args:
        k (int): 
        ldhdata : See `reappraisalmodel.ldhdata.LDHDataModule`
    """
    all_metrics = []
    
    max_epochs = trainer_kwargs.get('max_epochs', 20)

    for i in range(k):
        # Select the dataloaders for the given split.
        split = i
        train_dl = ldhdata.get_train_dataloader(split)
        val_dl = ldhdata.get_val_dataloader(split)
        
        callback_checkpoint = ModelCheckpoint(
            monitor='val_loss', verbose=False, 
            filename=f'best_model_{split}',
            save_last=False, save_top_k=1, save_weights_only=False, 
            mode='min', period=1, prefix=strat)

        model = LightningReapp(default_config)
        trainer = lit.Trainer(
            gpus = 1 if torch.cuda.is_available() else None,
            gradient_clip_val=1.0,
            progress_bar_refresh_rate=30,
            max_epochs=max_epochs,
            terminate_on_nan=True,
            early_stop_callback=True,
            callbacks=[callback_checkpoint],
            **trainer_kwargs)
        print(f"Training on split {i + 1}")
        trainer.fit(model, train_dl, val_dl)
        all_metrics.append(trainer.logged_metrics)
    return pd.DataFrame(all_metrics)

## Hyperparameter Tuning

Sources:
- [Scaling Up PyTorch Lightning Hyperparameter Tuning w/ Ray](https://medium.com/distributed-computing-with-ray/scaling-up-pytorch-lightning-hyperparameter-tuning-with-ray-tune-4bd9e1ff9929
)

In [ ]:
# export
from functools import partial
from argparse import ArgumentParser

import torch
import pytorch_lightning as lit
from ray.tune.integration.pytorch_lightning import TuneReportCallback
from ray import tune

from reappraisalmodel.lightningreapp import LightningReapp

parser = ArgumentParser()


callback_tuner = TuneReportCallback(
    {
        "loss": "val_loss",
        # "mean_accuracy": "val_accuracy"
    },
    on="validation_end",
)

default_tune_config = {
    "lr": tune.loguniform(1e-4, 1e-1), # loguniform samples by magnitude
    "hidden_layer_size": tune.quniform(10, 50)
}

### TUNING HYPERPARAMETERS
def train_tune(config, ldhdata, **tuner_kwargs):
    model = LightningReapp(config)

    max_epochs = tuner_kwargs.get('max_epochs', 10)
    trainer = lit.Trainer(
        num_folds=3,
        fast_dev_run=1,
        max_epochs=max_epochs,
        gpus= 1 if torch.cuda.is_available() else None,
        progress_bar_refresh_rate=30,
        callbacks=[callback_tuner],
    )
    trainer.fit(model, ldhdata)


tune.run(train_tune, config=default_tune_config, num_samples=2)

